In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

### Lidando com modelos complexos.

E se quisermos criar um modelo que envia um **subconjunto de atributos por um caminho curto**, **um subconjunto diferente de atributos (possivelmente sobreposto) por um caminho longo** ou profundo e que **possui uma saída adicional** (conforme mostrado na figura abaixo)?

<img src="../../../../figures/rna_com_multiplas_saidas.png" width="300px">

Nesse caso, uma solução é usar várias entradas (`inputs`) e múltiplas saídas (`outputs`).

Uma arquitetura com várias entradas possibilita que a rede neural aprenda tanto **padrões complexos (usando o caminho longo)** quanto **padrões simples (através do caminho curto)**.

Um caso onde múltiplas saídas são necessários ocorre quando queremos **localizar** e **classificar** um objeto em uma imagem. 

Esta é uma tarefa de regressão (encontrar as coordenadas do centro do objeto, bem como sua largura e altura) e uma tarefa de classificação.

Neste exemplo, vamos usar novamente a base de dados habitacional da Califórnia.

### Carregando o conjunto de dados para regressão. 

+ Vamos usar o conjunto de dados habitacional da Califórnia e criar um regressor com uma rede neural.
    + Esse conjunto possui 20640 exemplos e 8 atributos e 1 rótulo numéricos.
    + O rótulo é o valor médio de casas no estado da Califórnia expresso em centenas de milhares de dólares.
    + Para mais informações, acesse: https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset

+ Usamos a função `fetch_california_housing()` do Scikit-Learn para carregar os dados.

+ Depois de carregar os dados, dividimos em um conjunto de treinamento, um conjunto de validação e um conjunto de teste, e padronizamos todos os atributos.

In [3]:
# Baixa a base de dados.
housing = fetch_california_housing()

# Divide o conjunto total de exemplos em conjuntos de treinamento e teste.
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)

# Divide o conjunto de treinamento em conjuntos de treinamento (menor) e validação.
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

# Aplica padronização às matrizes de atributos.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

### Múltiplas entradas.

Neste exemplo vamos enviar 5 atributos pelo caminho curto (atributos 0 a 4, totalizando 5 atributos) e 6 atributos pelo caminho longo (atributos 2 a 7, totalizando 6 atributos).

In [4]:
# Instanciando dois objetos do tipo "Input".
input_A = keras.layers.Input(shape=[5], name="short_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")

### Camadas ocultas.

+ Em seguida, criamos uma camada densa oculta (`Dense`) com 30 neurônios e usando a função de ativação `ReLU`. 
    + Assim que ela é criada, a **chamamos como uma função**, passando a entrada (`input_B`). 
        + É por isso que essa API é chamada de API Funcional. 
    + Observe que estamos apenas dizendo ao Keras como ele deve conectar as camadas, nenhum dado está sendo processado ainda.


+ Na sequência, criamos uma segunda camada densa oculta e, novamente, a usamos como uma função. 
    + Observe, no entanto, que passamos a saída da primeira camada oculta.

In [5]:
hidden1 = keras.layers.Dense(30, activation="relu", name='hidden1')(input_B)
hidden2 = keras.layers.Dense(30, activation="relu", name='hidden2')(hidden1)

### Concatenando dados.

Em seguida, usamos a função `keras.layers.concatenate()`, que cria uma camada do tipo `Concatenate` e imediatamente a chama com as entradas fornecidas, i.e., entrada A (`input_A`) e a saída da segunda camada oculta (ver a figura).

In [6]:
concat = keras.layers.concatenate([input_A, hidden2])

### Múltiplas saídas.

Na sequência, criamos as camadas de saída, **cada uma com um único neurônio e nenhuma função de ativação**, e as chamamos como uma função, passando o resultado da concatenação e a saída da segunda camada oculta, respectivamente.

In [7]:
# Camada de saída com um único neurônio e ativação linear.
output = keras.layers.Dense(1, name="main_output")(concat)

# Camada de saída com um único neurônio e ativação linear.
aux_output = keras.layers.Dense(1, name="aux_output")(hidden2)

### Criando o modelo.

Por fim, criamos um objeto do tipo `Model` do API Keras, especificando quais entradas e saídas usar.

Como existem duas entradas e duas saída, precisamos passar uma lista com os respectivos objetos.

In [14]:
model = keras.models.Model(
    inputs=[input_A, input_B],
    outputs=[output, aux_output]
)

### Compilando o modelo.

+ Cada saída precisa de sua própria função de perda, portanto, quando compilarmos o modelo, devemos passar **uma lista de perdas**.
    + Se passarmos uma única função de perda, o Keras assumirá que a mesma função deve ser usada para todas as saídas. 


+ Por padrão, o Keras calcula todas as perdas e as soma para obter a perda final usada para treinamento. 


+ No entanto, nesse exemplo, nos preocupamos muito mais com a saída principal do que com a saída auxiliar, então damos um peso muito maior à perda da saída principal através do parâmetro `loss_weights`. 

In [9]:
model.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer='sgd')

### Separando os atributos e treinando o modelo.

+ Antes de treinarmos o modelo com o método `fit` devemos criar as matrizes de atributos para treinamento, validação e teste.

+ As matrizes de atributos A contém os atributos de 0 à 4 e as matrizes de atributos B, os atributos de 2 à 7.

+ Passamos um par de matrizes (`X_train_A`, `X_train_B`), uma por entrada, para o método `fit`.

+ Depois, precisamos fornecer os rótulos para cada saída. 

+ Neste exemplo, as duas saídas (principal e auxiliar) devem prever a mesma coisa, portanto, devem usar os mesmos rótulos. 
    + Então, passamos `[y_train, y_train]` para treinamento e `[y_valid, y_valid]` para validação.

In [10]:
# Separando os atributos.
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B   = X_test[:, :5],  X_test[:, 2:]

# Treinando o modelo.
history = model.fit([X_train_A, X_train_B], 
                    [y_train, y_train], 
                    epochs=20, 
                    validation_data=([X_valid_A, X_valid_B], [y_valid, y_valid])
                   )

Epoch 1/20
363/363 [==============================] - 3s 5ms/step - loss: 0.8602 - main_output_loss: 0.7458 - aux_output_loss: 1.8899 - val_loss: 5.9702 - val_main_output_loss: 6.3391 - val_aux_output_loss: 2.6507
Epoch 2/20
363/363 [==============================] - 1s 4ms/step - loss: 0.5736 - main_output_loss: 0.5210 - aux_output_loss: 1.0469 - val_loss: 1.4693 - val_main_output_loss: 1.3954 - val_aux_output_loss: 2.1342
Epoch 3/20
363/363 [==============================] - 1s 4ms/step - loss: 0.5078 - main_output_loss: 0.4649 - aux_output_loss: 0.8936 - val_loss: 1.8651 - val_main_output_loss: 1.9608 - val_aux_output_loss: 1.0039
Epoch 4/20
363/363 [==============================] - 1s 4ms/step - loss: 0.4619 - main_output_loss: 0.4275 - aux_output_loss: 0.7711 - val_loss: 0.4492 - val_main_output_loss: 0.4084 - val_aux_output_loss: 0.8163
Epoch 5/20
363/363 [==============================] - 1s 4ms/step - loss: 0.4417 - main_output_loss: 0.4131 - aux_output_loss: 0.6997 - val_loss

### Avaliando o modelo.

Quando avaliamos o modelo com o conjunto de teste, o Keras retornará a perda total, bem como todas as perdas por saída individual.

In [11]:
total_loss, main_loss, aux_loss = model.evaluate([X_test_A, X_test_B], [y_test, y_test])

162/162 [==============================] - 1s 3ms/step - loss: 0.3608 - main_output_loss: 0.3481 - aux_output_loss: 0.4756


### Realizando predições com o modelo.

Da mesma forma, o método `predict()` **retornará previsões para cada saída**.

Testamos com os 3 primeiros exemplos do conjunto de teste.

In [12]:
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

y_pred_main, y_pred_aux = model.predict([X_new_A, X_new_B])

1/1 [==============================] - 0s 240ms/step


In [13]:
for i in range(len(y_pred_main)):
    print('Actual: %1.3f - Predicted main: %1.3f - Predicted aux: %1.3f' % (y_test[i], y_pred_main[i], y_pred_aux[i]))

Actual: 0.477 - Predicted main: 0.362 - Predicted aux: 0.744
Actual: 0.458 - Predicted main: 1.532 - Predicted aux: 1.777
Actual: 5.000 - Predicted main: 3.509 - Predicted aux: 3.089
